In [ ]:
import pandas as pd
import csv
import os
import numpy as np
import re
import shutil

from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/Data/csv/")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def extract_hashtags(raw_hg):
    if raw_hg == '[]':
        return np.nan
    else:
        raw_hg = eval(raw_hg)
        hg_lst = []
        for i in raw_hg:
            hg_lst.append(i['text'])
        return hg_lst

In [ ]:
Mondt = {"02":29, "03":31, "04":30, "05":1}
path_txt = "/content/drive/MyDrive/Data/txt/"
path = "/content/drive/MyDrive/Data/"

count_all = 0
for key, value in Mondt.items():
    for day_i in range(1, value+1):
        if day_i < 10:
            day_j = '0' + f'{day_i}'
        else:
            day_j = f'{day_i}'

        f_name = 'geo_2020-'+ key + '-' + day_j

        df = pd.read_csv('hy_' + f_name + '.csv', header=None, dtype = str, encoding='utf-8-sig', engine='python')
        df.columns = ['created_at', 'id', 'full_text', 'hashtags', \
                                'location', 'geo', 'coordinates', 'place', \
                                'retweet_count', 'favorite_count', 'language']

        # extract the colomns: 
        #'created_at', 'id', 'full_text', 'hashtags', 
        #'retweet_count', 'favorite_count', 'language'
        df1 = df.iloc[:, [0, 1, 2, 3, 8, 9, 10]].copy() 
        df1['date'] = pd.to_datetime(df1.iloc[:, 0]).dt.date

        #set 'date' to the first place
        df2 = df1.iloc[:, [7, 1, 2, 3, 4, 5, 6]].copy() # resort the colomns
        df3 = df2.drop_duplicates(subset='id', keep="first")

        df4 = df3[df3['language'] == 'en'].copy()
        df4['hashtags'] = df4['hashtags'].apply(extract_hashtags)

        df5 = df4.iloc[:, [0, 1, 2, 3, 4, 5]].copy()
        df5.reset_index(drop=True, inplace=True)
        df5['id'] = df5['id'].apply(str)
        df5['id'] = df5['id'].apply(lambda r: r.strip())

        geo = pd.read_csv(path_txt + f_name +'.txt', sep='\n', header=None, dtype=str)
        geo['sp_txt'] = geo[0].str.replace(',', '\\', n = 1)
        geo[['id', 'geography']] = geo['sp_txt'].str.split('\\', expand=True)                              

        sp_geo = geo.iloc[:, [2,3]].copy()
        sp_geo['id'] = sp_geo['id'].apply(str)
        sp_geo.reset_index(drop=True, inplace=True)
        sp_geo['id'] = sp_geo['id'].apply(lambda r: r.strip())

        combine_csv = pd.merge(df5, sp_geo, on=['id'], how='left')

        combine_csv.to_csv('all_gb.csv', mode='a', header=None, index=False, encoding='utf-8-sig')

        count_all = count_all + combine_csv.shape[0]
shutil.move('all_gb.csv', path + 'all_gb.csv')
print(f'The total number of tweets from country code gb is {count_all}')